In [1]:
# For Supervised Machine Learning Models
from ML_Models.Supervised_Learning.deepfm import run_deepfm, tune_deepfm_with_cross_validation
from ML_Models.Supervised_Learning.neural_network import run_neural_network, tune_neural_network_with_cross_validation
from ML_Models.Supervised_Learning.random_forest import run_random_forest, tune_random_forest_with_cross_validation
from ML_Models.Supervised_Learning.xgb import run_xgboost, tune_xgboost_with_cross_validation


# For Collaborative Filtering
from ML_Models.Collaborative_Filtering.k_nearest_neighbor import run_item_based_knn_CF, run_user_based_knn_CF
from ML_Models.Collaborative_Filtering.matrix_factorization import run_user_based_mf_CF


# Importing Helper Functions
from helper_functions import ensemble_supervised, ensemble_unsupervised


import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
## Importing Necessary Files from Repository
x_train = pd.read_csv('./Data_Files/Training_Data/x_train.csv')
x_test = pd.read_csv('./Data_Files/Training_Data/x_test.csv')
y_train = pd.read_csv('./Data_Files/Training_Data/y_train.csv')
y_test = pd.read_csv('./Data_Files/Training_Data/y_test.csv')

to_predict = pd.read_csv('./Data_Files/Training_Data/unseen_movies.csv')
user_ratings = pd.read_csv('Data_Files/Raw_Data/users_ratings_img_20users_final.csv')

In [3]:
def train_and_tune_ML_Models(x_train, y_train, x_test, y_test):
    tune_deepfm_with_cross_validation(x_train, y_train, x_test, y_test)
    tune_neural_network_with_cross_validation(x_train, y_train, x_test, y_test)
    tune_random_forest_with_cross_validation(x_train, y_train, x_test, y_test)
    tune_xgboost_with_cross_validation(x_train, y_train, x_test, y_test)

In [4]:
## Training Supervised Models
train_and_tune_ML_Models(x_train, y_train, x_test, y_test)

100%|██████████| 20/20 [00:08<00:00,  2.47trial/s, best loss: 52.0056]
Fitting 3 folds for each of 72 candidates, totalling 216 fits
1260/1260 [==============================] - 2s 1ms/step
[CV 1/3] END activation_fn=tanh, batch_size=8, epochs=5, input_dimensions=(62,), num_neurons_1=20, num_neurons_2=20, optimizer_fn=adam;, score=-1.241 total time=  28.7s
1260/1260 [==============================] - 2s 2ms/step
[CV 2/3] END activation_fn=tanh, batch_size=8, epochs=5, input_dimensions=(62,), num_neurons_1=20, num_neurons_2=20, optimizer_fn=adam;, score=-1.235 total time=  28.4s
1260/1260 [==============================] - 2s 2ms/step
[CV 3/3] END activation_fn=tanh, batch_size=8, epochs=5, input_dimensions=(62,), num_neurons_1=20, num_neurons_2=20, optimizer_fn=adam;, score=-1.246 total time=  28.7s
1260/1260 [==============================] - 2s 2ms/step
[CV 1/3] END activation_fn=tanh, batch_size=8, epochs=5, input_dimensions=(62,), num_neurons_1=20, num_neurons_2=20, optimizer_fn=sg

In [9]:
def intelligent_recommender(to_predict, user_ratings):
    
    y_pred_dfm = run_deepfm(to_predict)
    y_pred_nn = run_neural_network(to_predict)
    y_pred_rf = run_random_forest(to_predict)
    y_pred_xgb = run_xgboost(to_predict)

    ## Ensembling Supervised Predictions
    ratings_df = ensemble_supervised(y_pred_dfm, y_pred_nn, y_pred_rf, y_pred_xgb, user_ratings)

    ## Generating Recommendations from Collaborative Filtering 
    recos_set_1 = run_item_based_knn_CF(ratings_df)
    recos_set_2 = run_user_based_knn_CF(ratings_df)
    recos_set_3 = run_user_based_mf_CF(ratings_df)

    ## Ensembling Unsupervised Recommendations
    consolidated_recos = ensemble_unsupervised(recos_set_1, recos_set_2, recos_set_3)

    return consolidated_recos

In [10]:
## Running Recommender System
intelligent_recommender(to_predict, user_ratings)

563/563 [==============================] - 3s 5ms/step
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'ur117926588': ['Edward Scissorhands',
  "The Simpsons - Lisa's First Word",
  'Castle in the Sky',
  'Toy Story',
  'The Last of Us - Long, Long Time',
  'Whiplash',
  'Zombieland',
  '21 Jump Street',
  'Rain Man',
  'Kill Bill: Vol. 2'],
 'ur4103165': ['Doctor Who - The End of Time: Part Two',
  "Doctor Who - Journey's End",
  'The Biggest Little Farm',
  'The Set-Up',
  'PK',
  'Pickup on South Street',
  'The Dissident',
  'Doctor Who - Last of the Time Lords',
  'Doctor Who - World Enough and Time',
  'Doctor Who - Doomsday'],
 'ur4532636': ['A Kind of Loving',
  "Wednesday - If You Don't Woe Me by Now",
  'The Justice of Bunny King',
  'Strangers on a Train',
  'Alice, Darling',
  'Bed & Board',
  'The Legend of Molly Johnson',
  'Sixteen Candles',
  'Happy-Go-Lucky',
  'High School'],
 'ur7813355': ['The Justice of Bunny King',
  'A Kind of Loving',
  'The Yards',
  "Wednesday - If You Don't Woe Me by Now",
  'Casino Royale',
  'La Luna',
  'Mamma Mia!',
  "Summer of '42",
  '